In [19]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses



### Uncomment below lines to download the data set

In [20]:
# url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"


# dataset = tf.keras.utils.get_file("aclImdb_v1", url,
#                                     untar=True, cache_dir='.',
#                                     cache_subdir='')

#dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [21]:
dataset = '.'

In [22]:
#dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
dataset_dir = os.path.join('aclImdb')


train_dir = os.path.join(dataset_dir,'train')
os.listdir(train_dir)


['labeledBow.feat',
 'neg',
 'pos',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [23]:
sample_file = os.path.join(train_dir,'pos/1181_9.txt') # show text data from the dataset
with open(sample_file) as f:
    print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [24]:
# remove_dir = os.path.join(train_dir,'unsup')
# shutil.rmtree(remove_dir)

In [25]:
batch_size = 32 
seed =42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(   # split training dataset to training set and validation set and get the training set
    'aclimdb/train',  
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [26]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

In [27]:
raw_val_ds=tf.keras.utils.text_dataset_from_directory( # get the validation set
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [28]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory( #get the text set
    'aclimdb/test',
    batch_size=batch_size
)

Found 25000 files belonging to 2 classes.


In [29]:
def custom_standardization(input_data): # remove punctuation and html tags for training
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [30]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization( #Tokanization and input embeddings
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [31]:
train_text=raw_train_ds.map(lambda x,y:x)
vectorize_layer.adapt(train_text)

In [32]:
def vectorize_text(text, label):
    text = tf.expand_dims(text,-1)
    print(text.shape.as_list())
    return vectorize_layer(text), label


In [33]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]

In [34]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

[None, 1]
[None, 1]
[None, 1]


In [36]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [38]:
embedding_dem = 16

In [40]:
model = tf.keras.Sequential([
    layers.Embedding(max_features,embedding_dem),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1)
])

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          160000    
                                                                 
 dropout_1 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160,017
Trainable params: 160,017
Non-trainable params: 0
__________________________________________________

In [43]:
model.compile(loss = losses.BinaryCrossentropy(from_logits=True),
              optimizer = 'adam',
              metrics = tf.metrics.BinaryAccuracy(threshold=0.0))

In [44]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs=epochs
)

Epoch 1/10
625/625 [==============================] - 225s 356ms/step - loss: 0.6655 - binary_accuracy: 0.6927 - val_loss: 0.6186 - val_binary_accuracy: 0.7670
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5528 - binary_accuracy: 0.7991 - val_loss: 0.5019 - val_binary_accuracy: 0.8220
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4482 - binary_accuracy: 0.8454 - val_loss: 0.4227 - val_binary_accuracy: 0.8458
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3797 - binary_accuracy: 0.8661 - val_loss: 0.3753 - val_binary_accuracy: 0.8592
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3367 - binary_accuracy: 0.8781 - val_loss: 0.3463 - val_binary_accuracy: 0.8668
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3069 - binary_accuracy: 0.8887 - val_loss: 0.3271 - val_binary_accuracy: 0.8716
Epoch 7/10
625/625 [==============================] - 2s 3ms

In [45]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 343s 433ms/step - loss: 0.3108 - binary_accuracy: 0.8721
Loss:  0.3108252286911011
Accuracy:  0.8721200227737427


### Export the model

In [47]:
export_model = tf.keras.Sequential([
    vectorize_layer,
    model,
    layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)

782/782 [==============================] - 318s 402ms/step - loss: 0.3108 - accuracy: 0.8721


In [51]:
examples = [
  "The movie was great!",
  "The movie was fine.",
  "The movie was terrible..."
]

export_model.predict(examples)

array([[0.5936632 ],
       [0.52397203],
       [0.33541337]], dtype=float32)